<font color='green'> 
**Youtube - Aladdin Persson Kanalı - TensorFlow 2.0 Beginner Tutorials serisi**
    
TensorFlow Tutorial 10 - Saving and Loading Models - Aladdin Persson anlattı.
</font>

**Video**: [TensorFlow Tutorial 10 - Saving and Loading Models](https://www.youtube.com/watch?v=idus3KO6Wic&list=PLhhyoLH6IjfxVOdVC1P1L5z5azs0XjMsb&index=10)

### İçindekiler

**Loading Dataset**

**Preprocessing Dataset**



### Kaynaklar

### <font color="blue"> Giriş</font>

Bu notebookta amaç modelin weightlerini ve tüm modeli nasıl save ve load edeceğimizi öğreneceğiz.

##### **1. How to save and load model weights** 

##### **2. Save and load entire model (serializing model)**
Tüm modeli kaydedip yüklerken, bir data structure olarak kaydedilecek ve bu, modelin tensorflow javascript, tensorflow lite gibi farklı tensorflow frameworklerine yüklenebileceği anlamına geliyor. Yani örneğin PC'nizde bir modeli eğitebilir ve diyelim ki onu üretime almak ve bir uygulama yapmak istiyorsunuz, daha önce eğittiğiniz modeli yükleyebilirsiniz ve herhangi bir dönüştürme yapmanıza gerek olmaz.

Tüm modeli yüklediğimizde:
* Weightler kaydedilecek. 
* Model architecture'ı (model mimarisi) kaydedilecek. Bu şekilde modelin koduna ihtiyacımız kalmayacak.
* Training configurationı da (eğitim yapılandırmasını) da kaydedecek, böylece model.compile() dosyasına göndereceğiniz şey bu olacaktır.
* Optimizer ve state'leri kaydedecek. For example if you're using the Adam optimizer it's gonna keep track of the exponential weighted averages and that's internal to the optimizer. 

Ve son olarak, optimize ediciyi ve durumları da kaydedecek. Örneğin, Adam optimize ediciyi kullanıyorsanız, üstel ağırlıklı ortalamaları takip edecektir ve bu, optimize edicinin içindedir.

- And then lastly it's also going to save the optimizer and states. So for example if you're using the Adam optimizer it's gonna keep track of the exponential weighted averages and that's internal to the optimizer. Örneğin, yalnızca weightleri kaydediyorsanız, modelden weightleri her yüklediğinizde optimize edici durumlar sıfırlanacaktır.


In [12]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

### 1. Loading Dataset

In [13]:
(x_train, y_train),(x_test, y_test) = mnist.load_data()

In [14]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


### 2. Preprocessing Dataset

In [15]:
x_train = x_train.reshape(-1, 28 * 28).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28*28).astype("float32") / 255.0

### 3. Creating Models

3 farklı model yazdık.

##### <font color="green">İlkinde Sequential API kullandık. </font>

In [16]:
model1 = keras.Sequential(
    [
     layers.Dense(64, activation='relu'),
     layers.Dense(10)
    ]
)

##### <font color="green">İkincisinde aynı modeli Functional API kullanarak yazdık. </font>

In [17]:
inputs = keras.Input(784)
x = layers.Dense(64, activation='relu')(inputs)
outputs = layers.Dense(10)(x)
model2 = keras.Model(inputs=inputs, outputs=outputs)

##### <font color="green">Üçüncüsünde modelimizi subclassing kullanarak yazdık. </font>

In [18]:
class MyModel(keras.Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.dense1 = layers.Dense(64, activation='relu')
    self.dense2 = layers.Dense(10)

  def call(self, input_tensor):
    x = tf.nn.relu(self.dense1(input_tensor))
    return self.dense2(x)

model3 = MyModel()

### 4. Specify Model Build From the Three API's

##### <font color="green">İlk modeli çalıştırıyoruz.</font>

In [19]:
model = model1

In [22]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(),
    metrics=["accuracy"]
)

In [23]:
model.fit(x_train, y_train, batch_size=32, epochs=2, verbose=2)

Epoch 1/2
1875/1875 - 6s - loss: 0.3107 - accuracy: 0.9123
Epoch 2/2
1875/1875 - 3s - loss: 0.1503 - accuracy: 0.9565


In [24]:
model.evaluate(x_test, y_test, batch_size=32, verbose=2)

313/313 - 1s - loss: 0.1286 - accuracy: 0.9620


[0.12860487401485443, 0.9620000123977661]

### 5. How to Save and Load Model Weights



In [25]:
model.save_weights('saved_model/') 

`model.save_weights('saved_model/')`: bir klasör adı yazıyoruz parantez içine. Notebookun bulunduğu yerde bu adda bir klasör açıyor. Scripti çalıştırdığımız yere baktığımızda saved_model adında bir klasör göreceğiz ve weightlerin kaydedildiği dosyalar bu klasörün içinde yer alıyor.

### 5. Save and Load Entire Model (Serializing Model)

- It will save the weights.
- It will save the model arhitecture.
- It will save the training configuration (model.compile())
- Optimizer and states

Tüm modeli kaydetmek istediğimizde data structure olarak kaydedecek. 